In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series
### The bad way

In [2]:
index = [('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
        ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
# all values from 2010
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The better way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [10]:
pop_df = pd.DataFrame({'total': pop,
                      'under18': [9267089, 9284094,
                                 4687374, 4318033,
                                 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [14]:
pop_df.unstack()

total            under18         
                2000      2010     2000     2010
California  33871648  37253956  9267089  9284094
New York    18976457  19378102  4687374  4318033
Texas       20851820  25145561  5906301  6879014

In [15]:
pop_df.unstack().unstack()

total    2000  California    33871648
               New York      18976457
               Texas         20851820
         2010  California    37253956
               New York      19378102
               Texas         25145561
under18  2000  California     9267089
               New York       4687374
               Texas          5906301
         2010  California     9284094
               New York       4318033
               Texas          6879014
dtype: int64

## Methods of MultiIndex Creation

In [16]:
df = pd.DataFrame(np.random.rand(4, 2),
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.379663  0.254296
  2  0.007995  0.241991
b 1  0.175956  0.205443
  2  0.206908  0.023665

In [18]:
data = {('California', 2000): 33871648,
       ('California', 2010): 37253956,
       ('Texas', 2000): 20851820,
       ('Texas', 2010): 25145561,
       ('New York', 2000): 18976457,
       ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

In [19]:
pd.MultiIndex.from_arrays([['a','a','b','b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [20]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [22]:
# Cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [23]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], 
             labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  


MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [24]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], 
             codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### MultiIndex level names

In [27]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex of columns

In [29]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                  names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names=['subject', 'type'])
# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  37.4  29.0  36.2  41.0  37.3
     2      33.0  39.0  54.0  36.3  40.0  37.0
2014 1      54.0  37.3  25.0  38.1  38.0  38.4
     2      51.0  36.4  38.0  37.4  28.0  34.6

In [30]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      29.0  36.2
     2      54.0  36.3
2014 1      25.0  38.1
     2      38.0  37.4

## Indexing and Slicing a MultiIndex
### Multiply indexed Series

In [31]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [32]:
pop['California', 2000]

33871648

In [33]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [34]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [35]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [36]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [37]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [38]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  37.4  29.0  36.2  41.0  37.3
     2      33.0  39.0  54.0  36.3  40.0  37.0
2014 1      54.0  37.3  25.0  38.1  38.0  38.4
     2      51.0  36.4  38.0  37.4  28.0  34.6

In [39]:
# Columns are primary in a DataFrame
health_data['Guido', 'HR']

year  visit
2013  1        29.0
      2        54.0
2014  1        25.0
      2        38.0
Name: (Guido, HR), dtype: float64

In [41]:
health_data['Guido', 'HR'][2013]

visit
1    29.0
2    54.0
Name: (Guido, HR), dtype: float64

In [42]:
health_data['Guido', 'HR'][2013, 2]

54.0

In [43]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      28.0  37.4
     2      33.0  39.0

In [44]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        28.0
      2        33.0
2014  1        54.0
      2        51.0
Name: (Bob, HR), dtype: float64

In [45]:
health_data.loc[(:, 1),(:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-45-22c7e58ea242>, line 1)

In [46]:
# get around this by building the desired slice explicitly using Pythoh's build in slice(), 
# but a better way in this context is to use an IndeSlice object.
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,28.0,29.0,41.0
2014,1,54.0,25.0,38.0


## Rearranging Multi-Indices
### Sorted and unsorted indices

In [47]:
# Many of the MultiIndex slicing operation will fail if the index is not sorted.
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names=['char', 'int']
data

char  int
a     1      0.684720
      2      0.228309
c     1      0.583823
      2      0.531306
b     1      0.674027
      2      0.316176
dtype: float64

In [48]:
# if we try to take a partial slice of this index, it will result in an error:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [49]:
data = data.sort_index()
data

char  int
a     1      0.684720
      2      0.228309
b     1      0.674027
      2      0.316176
c     1      0.583823
      2      0.531306
dtype: float64

In [50]:
data['a':'b']

char  int
a     1      0.684720
      2      0.228309
b     1      0.674027
      2      0.316176
dtype: float64

### Stacking and unstacking indices

In [51]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [52]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [53]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

In [58]:
# rearrange hierarchical data: turn the index labels into columns using reset_index
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [59]:
# raw input data looks like this, to build MultiIndex using set_index method of DataFrame
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

In [60]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  37.4  29.0  36.2  41.0  37.3
     2      33.0  39.0  54.0  36.3  40.0  37.0
2014 1      54.0  37.3  25.0  38.1  38.0  38.4
     2      51.0  36.4  38.0  37.4  28.0  34.6

In [61]:
# average out the measurements in the two visits each year
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     30.5  38.20  41.5  36.25  40.5  37.15
2014     52.5  36.85  31.5  37.75  33.0  36.50

In [63]:
# by further making use of the axis keyword
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,37.5,37.200000
2014,39.0,37.033333
